In [16]:
import re, json, pathlib, logging, time, argparse, pprint 
import datetime as dt
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator

import polmap.polmap as plmp


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
colors = ["#E5243B","#DDA63A","#4C9F38","#C5192D",
"#FF3A21","#26BDE2","#FCC30B","#A21942",
"#FD6925","#DD1367","#FD9D24","#BF8B2E",
"#3F7E44","#0A97D9","#56C02B","#00689D","#19486A"]

sdg_colors = {f'SDG {count.zfill(width)}' : color for count, color in enumerate(colors)}

bubble_chart = BubbleChart(area=total['Goal'],
                           bubble_spacing=0.1)

bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
bubble_chart.plot(
    ax, browser_market_share['Goal'], sdg_colors[browser_market_share['Goal']])
ax.axis("off")
ax.relim()
ax.autoscale_view()
ax.set_title('Browser market share')

plt.show()

In [ ]:
#Expand keywords

input = 'keywords/keywords_copia.xlsx'
keywords = pd.ExcelFile(input)
filename = pathlib.Path(input).stem
timestamp =  dt.datetime.now().isoformat(timespec='seconds').replace(':','').replace('T','_T')
filename += f'_{timestamp}'

output = f'{filename}.xlsx'

#sheets = a list of sheets where keywords are
#keys_df = [keywords.parse(sheet) for sheet in sheets]
keys_dfs = [keywords.parse('Target_keys'), keywords.parse('Goal_keys'), keywords.parse('MOI')]

for index, df in enumerate(keys_dfs):
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';;', ';', keywords))
    ex_df=df['Keys'].str.split(';', expand=True)
    lab_df=df.drop(columns='Keys')
    keys_dfs[index]=lab_df.join(ex_df)
    #df['Keys']=df['Keys'].apply(lambda keywords: re.sub(r'[^a-zA-Z0-9; -.]+', ';', keywords))

ex_df.head()

keys_dfs.append(keywords.parse('developing_countries'))

target_df, goal_df, devco_df, devco_names = keys_dfs

In [ ]:
target_df.head()

In [ ]:
ex_target_df=target_df['Keys'].str.split(';', expand=True)
lab_target_df=target_df.drop(columns='Keys')
lab_target_df.head()

In [ ]:
with pd.ExcelWriter(output, engine='xlsxwriter') as writer: # 'keywords_copy_for_transaltion.xlsx'
    target_df.to_excel(writer, sheet_name='Target_keys')
    goal_df.to_excel(writer, sheet_name='Goal_keys')
    devco_df.to_excel(writer, sheet_name='MOI')
    devco_names.to_excel(writer, sheet_name='developing_countries')

In [ ]:
#Load expand keywords

input = 'keywords/keywords_expanded.xlsx'

keywords = pd.ExcelFile(input)

sheets = keywords.sheet_names
print(f"Reading keywords dataset in {input}.\nSheets: {', '.join(sheets)}")

keywords = {sheet : keywords.parse(sheet, index_col=0) for sheet in sheets}
keywords_sheets = ['Target_keys', 'Goal_keys', 'MOI']

#remove all from stop_words to keep in keywords
stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
stop_words.remove('all')

for sheet in keywords_sheets:
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: [plmp.preprocess_text(keyword, stop_words) for keyword in keywords.split(';')])
    keywords[sheet].iloc[:,2:]=keywords[sheet].iloc[:,2:].applymap(lambda keyword: plmp.preprocess_text(str(keyword), stop_words))

In [ ]:
keywords['Target_keys'].head()

In [ ]:
#Load text

input = pathlib.Path('input/52019DC0640_stemmed')

files = sorted(input.glob('**/*.*'))

print(files)

text_ls = {}

for _file in files:
    with open(_file, 'r') as _f:
        text_ls[_file.stem]=_f.read()

In [ ]:
def count_keyword(text, keyword):

count = text.count(keyword)


In [ ]:
keywords['Target_keys']

In [ ]:
text_df = []

texts = list(text_ls.values())

keyword_matrix = keywords['Target_keys'].iloc[:,2:]#.to_numpy()
count_matrix = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: texts[0].count(keyword))#.to_numpy()
labels = keywords['Target_keys'].iloc[:,0]

counted_keywords = count_matrix.mask(count_matrix > 0, keyword_matrix)
counted_keywords = counted_keywords.apply(lambda keyword : ', '.join(keyword.astype(str)))
count_matrix = count_matrix.replace(0, '')
counted_keywords.to_excel('key_col.xlsx')
count_matrix.to_excel('mask.xlsx')


# for name, text in text_ls.items():
#     count_df = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: text.count(keyword))
#     #count_matrix = count_df#.to_numpy()
#     count_df = count_df.join(labels, how='inner')
#     count_df['Sum_counts'] = count_df.sum(axis=0)
#     keywords_ = keyword_matrix[count_matrix>0]
#     #count_df = count_df[count_df['Sum_counts'] > 0]
#     # #df.replace(0, None, inplace=True)
#     #df.insert(loc=1, column='Label', value=keywords['Target_keys'].iloc[:,1])
#     # df.insert(loc=2, coulmn='Count_keys', value=df.apply(lambda x: sum(x),axis=1))
#     # df.insert(loc=3, column='Diff_keys', value=df.apply(lambda x:count(x),axis=1))
#     count_df.to_excel(f'test_{name}.xlsx')
#     np.savetxt(f'test_{name}.csv', count_matrix, delimiter=',', fmt='%i')
#     np.savetxt(f'test_{name}_keys.csv', keywords_, delimiter=',',fmt='%s')
#     # np.savetxt('test.csv', count_matrix, delimiter=',')
#     text_df.append(count_df)

In [ ]:
keyword_matrix.head()

In [ ]:
count_matrix=count_matrix.mask(count_matrix > 1, -1)
count_matrix.to_excel('mask.xlsx')

In [24]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

['target_count', 'filtered_target_count', 'undetected_targets', 'goal_count', 'goal_overview', 'total_count_(goals_+_targets)']


In [37]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

Unnamed: 0  \
Policy                                             Goal   Target               
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 10 10.3            82   
                                                          10.b            88   
                                                   SDG 11 11.2            91   
                                                          11.3            92   
                                                          11.a            97   
...                                                                      ...   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 13 13.2           112   
                                                   SDG 15 15.5           130   
                                                   SDG 7  7.2             56   
                                                          7.3             57   
                                                   SDG 9  9.1             72   

                                                                  Target_ID  \
Policy                                             Goal   Target              
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 10 10.3           83   
                                                          10.b           89   
                                                   SDG 11 11.2           92   
                                                          11.3           93   
                                                          11.a           98   
...                                                                     ...   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 13 13.2          113   
                                                   SDG 15 15.5          131   
                                                   SDG 7  7.2            57   
                                                          7.3            58   
                                                   SDG 9  9.1            73   

                                                                  Target_Count  \
Policy                                             Goal   Target                 
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 10 10.3               1   
                                                          10.b               1   
                                                   SDG 11 11.2               6   
                                                          11.3               2   
                                                          11.a               1   
...                                                                        ...   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 13 13.2               4   
                                                   SDG 15 15.5               1   
                                                   SDG 7  7.2                1   
                                                          7.3                2   
                                                   SDG 9  9.1                1   

                                                                  Textlength  \
Policy                                             Goal   Target               
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 10 10.3         54614   
                                                          10.b         54614   
                                                   SDG 11 11.2         54614   
                                                          11.3         54614   
                                                          11.a         54614   
...                                                                      ...   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 13 13.2          3710   
                                                   SDG 15 15.5          3710   
                                                   SDG 7  7.2           3710   
                                                  

In [27]:
targets.head

,Unnamed: 0,Target,Goal,Target_ID,Policy,Count,Keyword,Textlength,Num_keys
0,7,2.1,SDG 2,8,52019DC0640/CELEX_52019DC0640_EN_TXT.html,2,food insecur - food all,54614,2
1,60,8.1,SDG 8,61,52019DC0640/CELEX_52019DC0640_EN_TXT.html,6,econom growth - inclus growth - sustain g...,54614,3
2,62,8.3,SDG 8,63,52019DC0640/CELEX_52019DC0640_EN_TXT.html,1,support sme,54614,1
3,130,15.5,SDG 15,131,52019DC0640/CELEX_52019DC0640_EN_TXT.html,10,biodivers loss - restor biodivers,54614,2
4,72,9.1,SDG 9,73,52019DC0640/CELEX_52019DC0640_EN_TXT.html,3,inland waterwai,54614,1


In [34]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()

Unnamed: 0  Count  \
Policy                                             Goal                        
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 12           0      5   
                                                   SDG 13           1     67   
                                                   SDG 15           2      2   
                                                   SDG 17           3      3   
                                                   SDG 3            4      2   
                                                   SDG 4            5      1   
                                                   SDG 7            6      2   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12           7      4   
                                                   SDG 13           8     63   
                                                   SDG 15           9      2   
                                                   SDG 17          10      3   
                                                   SDG 3           11      1   
                                                   SDG 4           12      1   
                                                   SDG 7           13      2   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12          14      1   
                                                   SDG 13          15      3   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12          16      4   
                                                   SDG 13          17     64   
                                                   SDG 15          18      2   
                                                   SDG 17          19      3   
                                                   SDG 3           20      2   
                                                   SDG 4           21      1   
                                                   SDG 7           22      2   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12          23      1   
                                                   SDG 13          24      3   

                                                           Textlength  \
Policy                                             Goal                 
52019DC0640/CELEX_52019DC0640_EN_TXT.html          SDG 12       54614   
                                                   SDG 13       54614   
                                                   SDG 15       54614   
                                                   SDG 17       54614   
                                                   SDG 3        54614   
                                                   SDG 4        54614   
                                                   SDG 7        54614   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12       52010   
                                                   SDG 13       52010   
                                                   SDG 15       52010   
                                                   SDG 17       52010   
                                                   SDG 3        52010   
                                                   SDG 4        52010   
                                                   SDG 7        52010   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12        3516   
                                                   SDG 13        3516   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12       51071   
                                                   SDG 13       51071   
                                                   SDG 15       51071   
                                                   SDG 17       51071   
                                                   SDG 3        51071   
                                                   SDG 4        51071   
                                                   SDG 7        51071   
52019DC0640/cellar_b828d165-1c22-11ea-8c1f-01aa... SDG 12        3710   
